In [1]:

from google.colab import drive
drive.mount("/content/drive/")


Mounted at /content/drive/


In [2]:

!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00


In [3]:

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [5]:
train_path = "/content/drive/MyDrive/Colab Notebooks/tweet_classification/train_val.csv"

In [6]:
test_path = "/content/drive/MyDrive/Colab Notebooks/tweet_classification/test.csv"

In [7]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [8]:
def count_distinct_strings(df):

    distinct_strings_set = {}
    for cell_value in df['labels']:
        strings_list = cell_value.split()
        for element in strings_list:
          if element in distinct_strings_set:
              distinct_strings_set[element] += 1
          else:
              distinct_strings_set[element] = 1
    return distinct_strings_set
label_count = count_distinct_strings(train_df)
print(label_count)

{'ingredients': 436, 'side-effect': 3805, 'mandatory': 783, 'rushed': 1477, 'ineffective': 1672, 'political': 626, 'none': 629, 'conspiracy': 487, 'country': 201, 'pharma': 1273, 'unnecessary': 722, 'religious': 64}


In [9]:
def split_and_store_distinct_strings(df):

    distinct_strings_set = set()
    max = 0
    min = 3
    for cell_value in df['labels']:

        strings_list = cell_value.split()
        if max<len(strings_list):
          max = len(strings_list)
        if min>len(strings_list):
          min = len(strings_list)
        distinct_strings_set.update(strings_list)
    print(max)
    print(min)
    return distinct_strings_set
all_labels = split_and_store_distinct_strings(train_df)

3
1


In [10]:
all_labels

{'conspiracy',
 'country',
 'ineffective',
 'ingredients',
 'mandatory',
 'none',
 'pharma',
 'political',
 'religious',
 'rushed',
 'side-effect',
 'unnecessary'}

In [11]:

def update_csv_with_new_columns(df, column_name, distinct_strings_set):
    for distinct_string in distinct_strings_set:
        df[distinct_string] = 0
    for index, row in df.iterrows():
        target_column_value = row[column_name]
        for distinct_string in distinct_strings_set:
            if distinct_string in target_column_value:
                df.at[index, distinct_string] = 1

    return df
train_df = update_csv_with_new_columns(train_df,"labels",all_labels)
train_df.head()

,ID,tweet,labels,ingredients,country,unnecessary,conspiracy,side-effect,ineffective,religious,none,rushed,political,mandatory,pharma
0,1296010336907038720t,@cath__kath AstraZeneca is made with the kidne...,ingredients,1,0,0,0,0,0,0,0,0,0,0,0
1,1336808189677940736t,It begins. Please find safe alternatives to th...,side-effect,0,0,0,0,1,0,0,0,0,0,0,0
2,1329488407307956231t,"@PaolaQP1231 Well, I mean congratulations Covi...",side-effect,0,0,0,0,1,0,0,0,0,0,0,0
3,1364194604459900934t,@BorisJohnson for those of us that do not wish...,mandatory,0,0,0,0,0,0,0,0,0,0,1,0
4,1375938799247765515t,She has been trying to speak out: writing lett...,side-effect rushed,0,0,0,0,1,0,0,0,1,0,0,0


In [12]:
train_df.drop(labels=['labels','ID'], axis=1, inplace=True)

In [13]:
target_list = list(all_labels)

In [14]:
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 15
LEARNING_RATE = 1e-05


In [15]:
from transformers import BertTokenizer, BertModel


In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [17]:
!pip install emoji
import re
import emoji
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['tweet']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        tweet = str(self.title[index])
        tweet = re.sub(r'http\S+', '', tweet)
        tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
        tweet = re.sub(r'[^A-Za-z0-9 ]+', '', tweet)

    # Convert emojis to text representation
        tweet = emoji.demojize(tweet)


        title = " ".join(tweet.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.8/361.8 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for emoji: filename=emoji-2.7.0-py2.py3-none-any.whl size=356563 sha256=e387e8e88eb2cccaf04eddec651145547ce07cd544e4a37d572dfc988df229ea
  Stored in directory: /root/.cache/pip/wheels/41/11/48/5df0b9727d5669c9174a141134f10304d1d78a3b89a4676f3d
Successfully built emoji


In [18]:

import re
import emoji
class CustomtestDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['tweet']
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        tweet = str(self.title[index])
        tweet = re.sub(r'http\S+', '', tweet)
        tweet = re.sub(r'@[A-Za-z0-9]+', '', tweet)
        tweet = re.sub(r'[^A-Za-z0-9 ]+', '', tweet)

    # Convert emojis to text representation
        tweet = emoji.demojize(tweet)


        title = " ".join(tweet.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
        }

In [19]:
from sklearn.model_selection import train_test_split

In [20]:

train_df = train_df.sample(frac=1, random_state=200).reset_index(drop=True)


train_size = 0.8
val_size = 0.2


train_rows = int(train_df.shape[0] * train_size)
val_rows = train_df.shape[0] - train_rows
# val_rows = int(train_df.shape[0] * val_size)
# test_rows = int(train_df.shape[0]*test_size)
train_df = train_df.head(train_rows)
val_df = train_df.tail(val_rows).reset_index(drop=True)
# val_df = val_df.head(val_rows)
# test_df = val_df.tail(test_rows).reset_index(drop=True)

In [21]:
train_df.shape

(7936, 13)

In [22]:
val_df.shape

(1985, 13)

In [23]:


train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [24]:

train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [25]:
test_dataset = CustomtestDataset(test_df, tokenizer, MAX_LEN)
test_data_loader = torch.utils.data.DataLoader(test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [26]:
train_data_iter = iter(train_data_loader)

# Fetch the first batch
batch = next(train_data_iter)

# Print the data in the first batch
print(batch)

{'input_ids': tensor([[  101,  2339,  2024,  ...,     0,     0,     0],
        [  101,  1996,  2061,  ...,     0,     0,     0],
        [  101,  2065,  2017,  ...,     0,     0,     0],
        ...,
        [  101,  4067, 15003,  ...,     0,     0,     0],
        [  101, 15624,  2715,  ...,     0,     0,     0],
        [  101,  2049,  9861,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'targets': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0

In [27]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


device

device(type='cuda')

In [28]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [29]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 12)

    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids,
            attention_mask=attn_mask,
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [30]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)


In [31]:

val_targets=[]
val_outputs=[]

In [32]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf


  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        #print('after loss data in training', loss.item(), train_loss)

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model


In [33]:
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/tweet_classification/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Colab Notebooks/tweet_classification/best_model.pt"


In [34]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      | 428815 KiB | 428815 KiB | 428815 KiB |      0 B   |\n|       from large pool | 428288 KiB | 428288 KiB | 428288 KiB |      0 B   |\n|       from small pool |    527 KiB |    527 KiB |    527 KiB |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         | 428815 KiB | 428815 KiB | 428815 KiB |      0 B   |\n|       from large pool | 428288 KiB | 428288 KiB |

In [35]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.001392 	Average Validation Loss: 0.004282
Validation loss decreased (inf --> 0.004282).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.001015 	Average Validation Loss: 0.003492
Validation loss decreased (0.004282 --> 0.003492).  Saving model ...
############# Epoch 2  Done   #############

############# Epoch 3: Training Start   #############
############# Epoch 3: Training End     #############
############# Epoch 3: Validation Start   #############
############# Epo

In [36]:
# Create an instance of your model and optimizer
# model = YourModelClass(...)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Load the trained model from the checkpoint
# def load_checkpoint(model, optimizer, checkpoint_path):
#     checkpoint = torch.load(checkpoint_path)
#     model.load_state_dict(checkpoint['state_dict'])
#     optimizer.load_state_dict(checkpoint['optimizer'])
#     return model, optimizer

# model, optimizer = load_checkpoint(model, optimizer, best_model_path)

def test_model(test_loader, model):

    model.eval()
    test_loss = 0
    all_predictions = []


    with torch.no_grad():
        for batch_idx, data in enumerate(test_loader, 0):
            ids = data['input_ids'].to(device, dtype=torch.long)
            mask = data['attention_mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)



            # Assuming you are using a sigmoid activation for multi-label classification
            predictions = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
            all_predictions.extend(predictions)



    # Convert predictions and targets to numpy arrays for evaluation
    all_predictions = np.array(all_predictions)
    return all_predictions


In [37]:
all_predictions= test_model(test_data_loader, model)

In [38]:
print(all_predictions)

[[0.00177736 0.0011679  0.00267041 ... 0.00190584 0.00165425 0.0033881 ]
 [0.0107258  0.00149787 0.00385688 ... 0.00374563 0.00351523 0.00623788]
 [0.00215541 0.0008328  0.00246501 ... 0.00183827 0.00218402 0.0058733 ]
 ...
 [0.00593859 0.00514013 0.00269281 ... 0.0055874  0.01286919 0.00238516]
 [0.00969012 0.01096834 0.00822842 ... 0.00949362 0.02105172 0.00519995]
 [0.00879729 0.00245558 0.00631663 ... 0.00778035 0.00704255 0.0466353 ]]


In [39]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 4.9 MB/s eta 0:00:00


In [41]:
all_predictions.shape

(486, 12)

In [78]:
all_predictions[0]

array([1.77736429e-03, 1.16790109e-03, 2.67040893e-03, 3.27085354e-03,
       9.91671324e-01, 3.38380411e-03, 7.89679354e-04, 1.87832979e-03,
       1.70097582e-03, 1.90584268e-03, 1.65425404e-03, 3.38810496e-03])

In [91]:
temp = []
result = []
temp.append(4)
result.append(temp)
print(result)

[[4]]


In [107]:
num_rows, num_cols = all_predictions.shape

for row in range(num_rows):
    temp = ""
    flag = 0
    for col in range(num_cols):
        if all_predictions[row, col] >= 0.5:
            if not temp:
              temp+=(target_list[col])
            else:
              temp+=(" ")
              temp+=(target_list[col])
            flag = 1
    if flag==0:
      if not temp:
        temp+=(target_list[np.argmax(all_predictions[row])])
      else:
        temp+=(" ")
        temp+=(target_list[np.argmax(all_predictions[row])])
    # result.append(temp)
    test_df.at[row,'pred_classes'] = temp





In [109]:
test_df.head()

,id,tweet,pred_classes
0,1070378532260470789t,Study Links HPV Vaccine to Historically High I...,side-effect
1,973746711964372993t,Deaths from tainted measles vaccine affecting ...,side-effect
2,1043031076787040257t,"Am apreciat un videoclip pe @YouTube, https://...",side-effect
3,1066338147527741440t,VIDEO --&gt;&gt; MMR #Vaccine increase risk of...,side-effect
4,963522018544152576t,Oral Polio Vaccine: Infecting Unvaccinated Kid...,side-effect


In [111]:
test_df.to_csv('answer.csv', index=False)
from google.colab import files
# Download the CSV file
files.download('answer.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>